In [1]:
import networkx as nx
import math
import random
import numpy as np

In [2]:
def euclidean_distance(a_x, a_y, b_x, b_y):
    return math.sqrt((a_x - b_x)**2 + (a_y - b_y)**2)

In [3]:
def read_graph(file_path):
    with open(file_path) as f:
        g = nx.Graph()
        for line in f:
            if not line[0].isdigit():
                continue
            node, x, y = line.split()
            node = int(node) - 1
            x = float(x)
            y = float(y)
            g.add_node(node, x=x, y=y)
        
        for u in g.nodes:
            for v in range(u + 1, len(g.nodes)):
                g.add_edge(u, v,
                           distance=euclidean_distance(g.nodes[u]['x'],
                                                        g.nodes[u]['y'],
                                                        g.nodes[v]['x'],
                                                        g.nodes[v]['y']))
        
        return g

In [4]:
graph = read_graph("wi29.tsp")

In [5]:
graph[0][1]["distance"]

74.53561415712697

In [6]:
graph.nodes

NodeView((0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28))

In [7]:
def foo(path, g):
    length = 0
    for node in range(len(path) - 1):
        length += g[node][node + 1]["distance"]
        print(node, node + 1, g[node][node+1]["distance"])
    length += g[path[len(path)-1]][path[0]]["distance"]
    return length

In [8]:
foo([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28], graph)

0 1 74.53561415712697
1 2 4069.7051490249282
2 3 1172.3669941144244
3 4 816.6666999999998
4 5 1533.3333000000002
5 6 3416.829291576882
6 7 1233.3333648648647
7 8 1586.4880519212525
8 9 3285.3631816825923
9 10 106.7186696152543
10 11 1411.95293791574
11 12 1314.2382487374398
12 13 1321.7628393130476
13 14 2350.1598668908555
14 15 3950.03516172817
15 16 2220.6731024123696
16 17 683.3332682926836
17 18 432.69187394470464
18 19 2174.409273291893
19 20 1399.5767871810572
20 21 370.5605639341698
21 22 220.14199962751314
22 23 3370.025289028461
23 24 620.2598281723953
24 25 1333.3333000000002
25 26 1400.396770760187
26 27 2023.542026854013
27 28 592.9399890054054


52284.07138604786

In [9]:
# kodiranje
# fitness
# ukrstanje
# selekcija
# mutacija

In [10]:
class Individual:
    def __init__(self, g):
        num_cities = len(g.nodes)
        self.code = np.random.permutation(num_cities).tolist()
        self.fitness = self.calc_fitness(g)
        
        
    def calc_fitness(self,g):
        length = 0
        n = len(self.code)
        for i in range(n-1):
            print(self.code[i], self.code[i+1])
            length += g[self.code[i]][self.code[i+1]]["distance"]
#         for node in range(len(self.code) - 1):
#             length += g[self.code[node]][self.code[node + 1]]["distance"]
# #             print(node, node + 1, g[node][node+1]["distance"])
#         g[self.code[len(self.code)-1]][self.code[0]]["distance"]
#         print(self.code[-1], self.code[0])
        length += g[self.code[-1]][self.code[0]]["distance"]
        return length
    
    def __lt__(self, other):
        return self.fitness < other.fitness

In [11]:
i = Individual(graph)

12 2
2 10
10 20
20 8
8 6
6 25
25 11
11 15
15 18
18 24
24 22
22 28
28 16
16 13
13 3
3 19
19 14
14 21
21 27
27 1
1 5
5 4
4 0
0 9
9 26
26 7
7 23
23 17


In [12]:
i.fitness

98411.25808494906

In [13]:
graph[27][28]["distance"]

592.9399890054054

In [14]:
len(i.code)

29

In [15]:
#TODO: benchmark algorithm, using vns for every individual, using vns only for the best individual of every generation
# compare those results

In [16]:
#for now, tournament selection
def selection(population, tournament_size,forbidden=None):
    allowed = list(set(range(len(population))).difference({forbidden}))
    chosen_indices = random.sample(allowed, tournament_size)
    best_fitness = float('inf')
    winner = None
    for index in chosen_indices:
        if best_fitness > population[index].fitness:
            winner = population[index]
            best_fitness = winner.fitness
    return winner

In [17]:
#1 3 4 5 6
#2 4 0 6 8
def crossover(parent1, parent2, child1, child2):
    n = len(parent1.code)
    breakpoint = random.randrange(n)
    child1.code[:breakpoint] = parent1.code[:breakpoint]
    child2.code[:breakpoint] = parent2.code[:breakpoint]
    #dodali smo breakpoint elemenata ostalo je da dodamo jos n - breakpoint
    # j mi krece od breakpointa
    # i mi krece od 0 i prolazi kroz sve gradove iz parent2
    # svaki put kada naidjem na grad koji nije u child1 na poziciji od 0 do breakpoint dodam ga na mesto j i j uvecam
    # takvih gradova postoji uvek n - breakpoint
    j = breakpoint
    for city in parent2.code:
        if city not in child1.code[:j+1]:
            child1.code[j] = city
            j+=1
    
    j = breakpoint
    for city in parent1.code:
        if city not in child2.code[:j+1]:
            child2.code[j] = city
            j+=1
#     j = breakpoint
#     k = breakpoint
#     for i in range(n):
#         if parent2.code[i] not in child1.code[:breakpoint]:
#             child1.code[j] = parent2.code[i]
#             j+=1
#         if parent1.code[i] not in child2.code[:breakpoint]:
#             child2.code[k] = parent1.code[i]
#             k+=1
#     if(breakpoint != 28 and child1.code[breakpoint] == child1.code[breakpoint+1]):
#         raise ValueError
#     if(breakpoint != 28 and child2.code[breakpoint] == child2.code[breakpoint+1]):
#         raise ValueError

In [18]:
def mutation(child, mutation_prob):
    
    p = random.random()
    # try using while instead of if
    if p <= mutation_prob:
        n = len(child.code)
        random_index1 = random.randrange(n)
        allowed = list(set(range(n)).difference({random_index1}))
        random_index2 = random.choice(allowed)
        child.code[random_index1], child.code[random_index2] = child.code[random_index2], child.code[random_index1]
    

In [19]:
def ga(g,num_iters, use_elitism, elitism_size, mutation_prob, tournament_size, population_size):
    #init population
    population = [Individual(g) for _ in range(population_size)]
    new_population = [Individual(g) for _ in range(population_size)]
    
    for _ in range(num_iters):
        if use_elitism:
            population.sort()
#             print()
#             print([population[i].fitness for i in range(population_size)])
#             print()
            new_population[:elitism_size] = population[:elitism_size]
        for i in range(elitism_size, population_size, 2):
            #selection
            parent1 = selection(population, tournament_size)
            parent2 = selection(population, tournament_size, parent1)
            
            child1 = new_population[i]
            child2 = new_population[i+1]
            
            #crossover
            crossover(parent1, parent2, child1, child2)
            #mutation
            mutation(child1, mutation_prob)
            mutation(child2, mutation_prob)
            #calc_new_fitnesses
            print(f"iter {i} parent 1 {parent1.code}")
            print(f"iter {i} parent 2 {parent2.code}")
            print(f"iter {i} child 1 {child1.code}")
            child1.fitness = child1.calc_fitness(g)
            print(f"iter {i} parent 1 {parent1.code}")
            print(f"iter {i} parent 2 {parent2.code}")
            print(f"iter {i} child 2 {child2.code}")
            child2.fitness = child2.calc_fitness(g)
            #vns
            #TODO
            #replace_population
            population[:] = new_population[:]

    #find solution
    best_individual = min(population)
    print(f"code: {best_individual.code}")
    print(f"fitness: {best_individual.fitness}")
    
    

In [20]:

ga( graph,
    num_iters = 100,
    use_elitism = True,
    population_size = 100,
    elitism_size = 100 // 5,
    mutation_prob = 0.1,
    tournament_size = 8,
    )

19 23
23 1
1 24
24 14
14 10
10 9
9 17
17 0
0 20
20 7
7 2
2 5
5 6
6 11
11 8
8 21
21 16
16 22
22 18
18 13
13 15
15 28
28 25
25 4
4 12
12 26
26 3
3 27
25 14
14 26
26 4
4 15
15 2
2 17
17 22
22 27
27 16
16 1
1 13
13 7
7 19
19 10
10 9
9 24
24 21
21 0
0 20
20 5
5 6
6 23
23 28
28 8
8 11
11 12
12 18
18 3
4 7
7 22
22 25
25 13
13 18
18 28
28 3
3 0
0 14
14 2
2 19
19 24
24 5
5 26
26 12
12 15
15 6
6 23
23 1
1 27
27 11
11 20
20 21
21 10
10 9
9 16
16 8
8 17
28 18
18 11
11 14
14 27
27 23
23 24
24 4
4 6
6 15
15 17
17 3
3 7
7 26
26 10
10 21
21 2
2 20
20 8
8 25
25 0
0 13
13 12
12 16
16 1
1 19
19 9
9 5
5 22
14 22
22 6
6 5
5 28
28 19
19 23
23 12
12 3
3 4
4 7
7 10
10 2
2 21
21 26
26 11
11 25
25 1
1 13
13 27
27 8
8 9
9 20
20 0
0 18
18 16
16 17
17 15
15 24
26 25
25 21
21 24
24 4
4 13
13 15
15 17
17 11
11 9
9 5
5 27
27 16
16 22
22 18
18 28
28 14
14 0
0 7
7 1
1 12
12 2
2 6
6 8
8 10
10 19
19 23
23 20
20 3
8 19
19 4
4 16
16 11
11 20
20 27
27 22
22 5
5 15
15 14
14 1
1 18
18 23
23 28
28 24
24 3
3 12
12 13
13 9
9 6
6

KeyError: 25